In [25]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [26]:
DATA_PATH = 'assets/crypto-data.csv'
SEQUENCE_LENGTH = 30

# Wczytanie danych

In [27]:
df = pd.read_csv(DATA_PATH)
df['real-daily-trend'] = df['monthly-btc-trend'] * df['daily-btc-trend'] / 100

In [28]:
df.head()

,date,open,high,low,close,volume,avg-block-size,n-transactions-per-block,n-payments-per-block,transactions-per-second,blocks-size,hash-rate,difficulty,monthly-btc-trend,daily-btc-trend,real-daily-trend
0,2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,1.029120e+10,1.043383,1549.724359,3152.608974,2.630903,150110.786114,1.497558e+07,1.922581e+12,81,45,36.45
1,2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,1.684660e+10,1.041368,1994.035088,4026.590643,3.848264,150273.540487,1.641554e+07,1.931136e+12,81,53,42.93
2,2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,1.687190e+10,1.065513,2522.057325,4718.490446,4.486979,150451.629839,1.507158e+07,1.931136e+12,81,56,45.36
3,2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,2.178320e+10,1.065833,2529.809524,5122.785714,5.275347,150618.904617,1.612755e+07,1.931136e+12,81,54,43.74
4,2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,2.384090e+10,1.067636,2396.552448,5124.125874,4.563021,150797.951213,1.372762e+07,1.931136e+12,81,48,38.88


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2647 entries, 0 to 2646
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      2647 non-null   object 
 1   open                      2647 non-null   float64
 2   high                      2647 non-null   float64
 3   low                       2647 non-null   float64
 4   close                     2647 non-null   float64
 5   volume                    2647 non-null   float64
 6   avg-block-size            2647 non-null   float64
 7   n-transactions-per-block  2647 non-null   float64
 8   n-payments-per-block      2647 non-null   float64
 9   transactions-per-second   2647 non-null   float64
 10  blocks-size               2647 non-null   float64
 11  hash-rate                 2647 non-null   float64
 12  difficulty                2647 non-null   float64
 13  monthly-btc-trend         2647 non-null   int64  
 14  daily-bt

In [30]:
df.describe()

,open,high,low,close,volume,avg-block-size,n-transactions-per-block,n-payments-per-block,transactions-per-second,blocks-size,hash-rate,difficulty,monthly-btc-trend,daily-btc-trend,real-daily-trend
count,2647.000000,2647.000000,2647.000000,2647.000000,2.647000e+03,2647.000000,2647.000000,2647.000000,2647.000000,2647.000000,2.647000e+03,2.647000e+03,2647.000000,2647.000000,2647.000000
mean,30281.863739,30941.148166,29581.103916,30306.100149,2.754548e+10,1.289727,2292.424912,4699.346216,3.686948,370809.060640,2.484452e+08,3.421812e+13,35.455988,64.631281,21.651632
std,24791.697635,25312.633506,24235.637231,24808.859147,2.004631e+10,0.327854,826.617653,1613.591151,1.237448,143268.936291,2.243894e+08,3.066762e+13,18.909865,18.560845,11.469697
min,3236.274658,3275.377930,3191.303467,3236.761719,2.923670e+09,0.431452,834.129630,1520.088889,1.554340,150110.786114,1.372762e+07,1.922581e+12,12.000000,17.000000,6.200000
25%,9165.169434,9308.080078,8976.270508,9167.385742,1.414231e+10,1.054804,1773.363020,3540.249883,2.961892,245931.117534,8.885100e+07,1.287580e+13,20.000000,52.000000,13.320000
50%,23240.203125,23739.138672,22861.558594,23264.291016,2.439725e+10,1.260745,2107.352000,4422.271523,3.420660,359529.874091,1.569433e+08,2.145523e+13,30.000000,66.000000,19.500000
75%,44960.826172,46619.900391,43783.865234,45050.421875,3.604767e+10,1.584134,2492.410661,5572.578759,3.965799,486896.703422,3.617916e+08,5.064621e+13,44.000000,78.000000,25.705000
max,106147.296875,109114.882812,105291.734375,106146.265625,3.509679e+11,2.524785,5517.916667,10476.342466,10.855556,648806.373469,9.888068e+08,1.141673e+14,100.000000,100.000000,100.000000


# Przygotowanie danych

In [31]:
df = df.drop(columns=['date'])
df = df.drop(columns=['monthly-btc-trend'])
df = df.drop(columns=['daily-btc-trend'])

In [32]:
n_samples = len(df) - SEQUENCE_LENGTH
n_features = df.shape[1]

In [33]:
X_full = np.zeros((n_samples, SEQUENCE_LENGTH, n_features), dtype=np.float64)
y_full = np.zeros((n_samples, 2), dtype=np.float64) 

for i in range(n_samples):
    X_full[i] = df.iloc[i:i+SEQUENCE_LENGTH].values
    y_full[i] = df.iloc[i+SEQUENCE_LENGTH][['high', 'low']].values

In [34]:
print(f"X={X_full.shape}")
print(f"y={y_full.shape}")

X=(2617, 30, 13)
y=(2617, 2)


Podzielmy dane na:
- train (60%)
- validation (20%)
- test (20%)

In [35]:
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y_full, test_size=0.4, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

print(f"Train:      X={X_train.shape}, y={y_train.shape}")
print(f"Validation: X={X_val.shape},   y={y_val.shape}")
print(f"Test:       X={X_test.shape},  y={y_test.shape}")

Train:      X=(1570, 30, 13), y=(1570, 2)
Validation: X=(523, 30, 13),   y=(523, 2)
Test:       X=(524, 30, 13),  y=(524, 2)
